In [1]:
using Pkg

Pkg.activate("../Project.toml")

using Random
import .Iterators: cycle, Cycle, take

using Knet
using Knet: deconv4, conv4, unpool, pool, mat, sigm, KnetArray, nll, zeroone, progress, adam!, sgd!, param, param0, dropout, relu, minibatch, Data
import Knet: train!
using JLD2
using CUDA

 Activating environment at `~/yolo-julia/Project.toml`


In [2]:
function Knet.KnetArray(x::CuArray{T,N}) where {T,N}
    p = Base.bitcast(Knet.Cptr, pointer(x))
    k = Knet.KnetPtr(p, sizeof(x), Int(CUDA.device().handle), x)
    KnetArray{T,N}(k, size(x))
end

In [3]:
include("../coco2014.jl")
x, y = load_data(
    "../dataset/images/overfit",
    "../val_labels.json",
    class_file="../class_mappings.json"
);

dtrn = minibatch(x[:, :, :, 1:6], y[1:6], 1; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(x[:, :, :, 10:11], y[10:11], 1; xsize = (416,416,3,:), xtype=Knet.atype());

xx, yy = first(dtrn)

(K32(416,416,3,1)[0.654902⋯], Any[Any[Any[4.0, 0.497327, 0.511852, 0.948637, 0.952609], Any[4.0, 0.240637, 0.217617, 0.475398, 0.424859]]])

In [65]:
include("../models.jl")

Random.seed!(1)
darknet = Darknet("../yolov3.cfg"; verbose=false);

a = darknet(xx, yy)
@diff a = darknet(xx, yy)


T(8.465776634216308)

In [56]:
min.(max.(exp.(ps[3:4, :]), 0), 1000)

2×11 KnetArray{Float32,2}:
 0.894371  0.894262  0.628493  0.658711  …  1.00179   0.760402  0.781808
 1.01587   1.02907   1.0577    1.04478      0.919599  0.973357  1.02486

In [38]:
j = convert(Array{Integer}, j)
size(j)
j = reshape(j, size(j))

3×6 Array{Integer,2}:
 0  0  1  1  1  1
 1  0  1  1  1  1
 1  0  0  0  1  0

In [39]:
an = convert(Array{Bool}, a)

3×6 Array{Bool,2}:
 0  0  1  1  1  1
 1  0  1  1  1  1
 1  0  0  0  1  0

In [19]:
vcat(an'...)

18-element Array{Int64,1}:
 0
 0
 1
 1
 1
 1
 1
 0
 1
 1
 1
 1
 1
 0
 0
 0
 1
 0

In [4]:
function train!(model, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
        println("Iter: ", i, " ", train_loss[end])
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end



train! (generic function with 2 methods)

In [9]:

dtrn = minibatch(x[:, :, :, 1:6], y[1:6], 1; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(x[:, :, :, 10:11], y[10:11], 1; xsize = (416,416,3,:), xtype=Knet.atype());
include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);

model = darknet

iters, trnloss, trnacc, tstloss, tstacc = train!(
    model, dtrn, dtst; 
    period=1, iters=520, lr=1e-2, optimizer=adam!);

Iter: 0 5.566567887067795
Iter: 

1 12.266344351172448
Iter: 2 12.674455112218858
Iter: 3 14.040614753961565
Iter: 4 15.708545681238176
Iter: 5 15.745373417139055
Iter: 6 15.836422921419144
Iter: 7 14.924522615671158
Iter: 8 14.907230426073076
Iter: 9 14.907230426073076
Iter: 10 13.886420694589617
Iter: 11 14.161012872457505
Iter: 12 13.62393023610115
Iter: 13 14.327960501909258
Iter: 14 13.88627369761467
Iter: 15 13.88627369761467
Iter: 16 13.886273697614671
Iter: 17 12.719603861570357
Iter: 18 12.17590509533882
Iter: 19 11.940695165395738
Iter: 20 11.728097308874132
Iter: 21 10.832674449682235
Iter: 22 10.427667962312698
Iter: 23 11.060097478628158
Iter: 24 10.428792876005174
Iter: 25 11.060283721685408
Iter: 26 11.025889242887496
Iter: 27 10.862514132261277
Iter: 28 10.862514132261277
Iter: 29 10.93624584555626
Iter: 30 11.203868960142136
Iter: 31 12.76885741353035
Iter: 32 12.880781050920488
Iter: 33 12.948065434694291
Iter: 34 12.503555866479877
Iter: 35 12.655064593553545
Iter: 36 12.72042605996132
Iter: 37 12.79

Iter: 295 5.122549914717674
Iter: 296 5.116064112782478
Iter: 297 5.063426078557968
Iter: 298 5.070132024288178
Iter: 299 5.102199027538299
Iter: 300 5.16163240134716
Iter: 301 5.116865597963334
Iter: 302 5.257165207862854
Iter: 303 5.500084355473518
Iter: 304 5.482650549411773
Iter: 305 5.407738534808158
Iter: 306 5.423487115502358
Iter: 307 5.475667032599449
Iter: 308 5.481200413703919
Iter: 309 5.521418715715408

Stacktrace:
 [1] Box at ./boot.jl:340 [inlined]
 [2] AutoGrad.Result(::KnetArray{Float32,4}, ::Function, ::Tuple{typeof(max),AutoGrad.Result{KnetArray{Float32,4}},AutoGrad.Result{KnetArray{Float32,4}}}, ::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /home/ec2-user/.julia/packages/AutoGrad/VFrAv/src/core.jl:111
 [3] forw(::Function, ::Function, ::Vararg{Any,N} where N; kwargs::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /home/ec2-user/.julia/packages/AutoGrad/VFrAv/src/core.jl:72
 [4] forw at /home/ec2-user/.julia/pack

LoadError: InterruptException:

In [22]:
@load "first.jld2" darknet

1-element Array{Symbol,1}:
 :darknet

In [23]:
@diff a = darknet(x[:, :, :, 1:2], y[1:2])

T(5.974961309897038)

In [ ]:
plot(iters, trnloss, label="train", xlabel="Iterations", ylabel="Loss")
display(plot!(iters, tstloss, label="test"))

plot(iters, round.(1 .- trnacc, digits=2), label="train", xlabel="Iterations", ylabel="Misclassification error")
display(plot!(iters, round.(1 .- tstacc, digits=2), label="test"))

In [ ]:

include("../models.jl")
Random.seed!(1)

darknet = Darknet("../yolov3.cfg"; verbose=false);

l = darknet(xx, yy)


In [ ]:
sum(1.0 .- giou)

In [ ]:
inter = clamp.(min.(b1_x2, b2_x2) - max.(b1_x1, b2_x1), 0, 1e10) .* 
clamp.(min.(b1_y2, b2_y2) - max.(b1_y1, b2_y1), 0, 1e10);


In [ ]:
pbox[1, :] .- pbox[3, :] ./ 2

In [ ]:
clamp.(exp.(scores), 0, 1e3)

In [ ]:

b = Integer.(t[:, 1])
c = Integer.(t[:, 2])
gxy = t[:, 3:4]
gwh = t[:, 5:6]

In [ ]:
size(yolo_out[1])

In [ ]:
j

In [ ]:
j'

In [ ]:
tt[jj, :]

In [ ]:
tt = repeat(t, 3)

In [ ]:
tt[j]

In [ ]:
at'[jj]

In [ ]:
targets_reshaped .* gain

In [ ]:
wh1r = reshape(wh1, (:, 1, 2))

In [ ]:
wh12 = reshape(prod(wh1r, dims=3), (:, 1))

In [ ]:
wh2

In [ ]:
wh2r = reshape(wh2, (1, :, 2))

In [ ]:
wh22 = reshape(prod(wh2r, dims=3), (1, :))

In [ ]:
wh12 .+ wh22

In [ ]:
reshape(prod(a, dims=3), (3, 13))

In [ ]:
inter = build_targets(yolo_out, yy, darknet)


In [ ]:
targets = nothing 

for (i, t) in enumerate(yy[2:end])
    println(i)

end
gg = hcat(yy[2]...)
# gg = vcat(ones(1, 2), gg)

In [ ]:
length(yy[2])


In [ ]:
include("utils/utils.jl")

